In [22]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, utils
from torch.utils.data import DataLoader
from PIL import Image
from tqdm import tqdm

In [23]:
save_dir = 'data/oxford_flowers'
os.makedirs(save_dir, exist_ok=True)

In [24]:
image_size = 64
batch_size = 128
latent_dim = 100
num_epochs = 1000
save_interval = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
os.makedirs("generated_flowers", exist_ok=True)

In [26]:
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  # For RGB
])

In [27]:
dataset = datasets.Flowers102(root='./data', split='train', download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [28]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512), nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256), nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128), nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64), nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    def forward(self, x): return self.net(x)

In [29]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128), nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256), nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512), nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x): return self.net(x).view(-1)

In [30]:
netG = Generator().to(device)
netD = Discriminator().to(device)

criterion = nn.BCELoss()
optimizerG = optim.Adam(netG.parameters(), lr=3e-4, betas=(0.5, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=3e-4, betas=(0.5, 0.999))

fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

In [31]:
for epoch in range(1, num_epochs + 1):
    for i, (real_imgs, _) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch}/{num_epochs}")):
        real_imgs = real_imgs.to(device)
        b_size = real_imgs.size(0)
        real_labels = torch.ones(b_size, device=device)
        fake_labels = torch.zeros(b_size, device=device)

        # Train Discriminator
        netD.zero_grad()
        output_real = netD(real_imgs)
        lossD_real = criterion(output_real, real_labels)

        noise = torch.randn(b_size, latent_dim, 1, 1, device=device)
        fake_imgs = netG(noise)
        output_fake = netD(fake_imgs.detach())
        lossD_fake = criterion(output_fake, fake_labels)

        lossD = lossD_real + lossD_fake
        lossD.backward()
        optimizerD.step()

        # Train Generator
        netG.zero_grad()
        output = netD(fake_imgs)
        lossG = criterion(output, real_labels)
        lossG.backward()
        optimizerG.step()

    # Save generated images after every 20 epochs
    if epoch % save_interval == 0:
        with torch.no_grad():
            fake_samples = netG(fixed_noise).detach().cpu()
        utils.save_image(
            fake_samples,
            f"generated_flowers/epoch_{epoch}.png",
            normalize=True,
            nrow=8
        )
        print(f"Saved generated image at epoch {epoch}!")

Epoch 1/1000:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 20/1000: 100%|██████████| 8/8 [00:09<00:00,  1.22s/it]


Saved generated image at epoch 20!


Epoch 40/1000: 100%|██████████| 8/8 [00:09<00:00,  1.23s/it]


Saved generated image at epoch 40!


...

Saved generated image at epoch 960!


Epoch 980/1000: 100%|██████████| 8/8 [00:09<00:00,  1.25s/it]


Saved generated image at epoch 980!


Epoch 1000/1000: 100%|██████████| 8/8 [00:09<00:00,  1.24s/it]


Saved generated image at epoch 1000!
